Эксперименты с metadata моделями

In [74]:
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Set
from collections import Counter
import Levenshtein
import matplotlib.pyplot as plt
from src import future_extract as fe
from src import text_model as tm
import matplotlib as mpl
import seaborn as sns


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
mpl.style.use('ggplot')
sns.set_style('white')

In [86]:
from src import preproces_text as pt
import importlib
importlib.reload(pt)

<module 'src.preproces_text' from '/Users/vladislavdemin/PycharmProjects/BB-E-CUP/src/preproces_text.py'>

In [47]:
path_train = '/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_train.csv'
path_test = '/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_test.csv'

In [48]:
data_raw, data_val, data_raw_full, data_val_full = pt.load_and_fill_data(path_train, path_test)

In [49]:
data_raw_with_features, data_val_with_features = pt.extract_features(data_raw, data_val)

In [50]:
data_raw_with_features_and_embed, data_val_with_features_and_embed = pt.vectoriz_text(data_raw_with_features, data_val_with_features)

Initialized TF-IDF + SVD (fast mode)
Processing DataFrame with 197198 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 1972/1972 [00:40<00:00, 48.85it/s]


Created 100 embedding columns
Final DataFrame shape: (197198, 148)
Processing DataFrame with 22760 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 228/228 [00:05<00:00, 40.44it/s]


Created 100 embedding columns
Final DataFrame shape: (22760, 147)


,id,description,brand_name,CommercialTypeName4,resolution,desc_has_html_tags,desc_html_br_count,desc_html_p_count,desc_html_div_count,desc_html_span_count,...,desc_ellipsis_count,desc_has_emoji,desc_paragraph_count,desc_avg_paragraph_length,desc_word_uniqueness,desc_repeated_words_count,desc_max_word_repetition,desc_repeated_bigrams_count,desc_text_entropy,desc_repeated_sentences
0,159385,"Мешки пылесборники для пылесоса PHILIPS, 10 шт...",ACTRUM,Пылесборник,0,0,0,0.0,0.0,0.0,...,0.0,0,1.0,886.0,0.650000,1,34.0,0.0,4.874119,0
1,288616,Защитная силиконовая крышка обьектива GoPro He...,Red Line,Крышка для объектива,0,1,2,0.0,0.0,0.0,...,0.0,0,1.0,388.0,0.867925,1,3.0,0.0,5.449526,0
2,108090,Плоский медиатор из кости толщиной 0.6 мм<br/>...,Talwar Brothers,Аксессуар для музыкального инструмента,0,1,1,0.0,0.0,0.0,...,0.0,0,1.0,218.0,0.757576,1,3.0,4.0,4.536670,0
3,415607,"Игра Sonic Frontiers для PlayStation 5, русски...",Unknown,Видеоигра,0,0,0,0.0,0.0,0.0,...,0.0,0,5.0,111.8,0.913043,1,3.0,0.0,5.923671,0
4,332391,Disney Classic Games: Aladdin and The Lion Kin...,Unknown,Видеоигра,0,0,0,0.0,0.0,0.0,...,0.0,0,1.0,540.0,0.871429,1,3.0,1.0,5.861356,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197193,159865,Парогенератор Tefal Express Vision SV8152E0 об...,Tefal,Паровая станция,0,0,0,0.0,0.0,0.0,...,0.0,0,1.0,582.0,0.880952,1,3.0,0.0,6.145235,0
197194,299636,Выпрямитель для волос Dyson Airstrait<br/><br/...,Dyson,Выпрямитель для волос Dyson,0,1,10,0.0,0.0,0.0,...,0.0,0,1.0,279.0,0.911765,0,2.0,0.0,4.910992,0
197195,24145,Настольная компактная посудомоечная машина Wei...,Weissgauff,Посудомоечная машина компактная,0,1,2,0.0,0.0,0.0,...,0.0,0,1.0,546.0,0.882353,2,4.0,2.0,5.793345,0
197196,51939,Беспроводные наушники Redmi Buds 4 Lite Молоде...,Xiaomi,Наушники TWS,1,1,6,0.0,0.0,0.0,...,0.0,0,1.0,584.0,0.866667,1,3.0,3.0,5.952087,0


In [65]:
from category_encoders import CatBoostEncoder
target_col = "resolution"  # укажи свою целевую колонку
cat_cols = ["brand_name", "CommercialTypeName4"]  # какие категориальные кодируем
cbe = CatBoostEncoder(cols=cat_cols, random_state=42)

cbe.fit(data_raw_with_features_and_embed[cat_cols], data_raw_with_features_and_embed[target_col])

data_raw_enc = data_raw_with_features_and_embed.copy()
data_val_enc = data_val_with_features_and_embed.copy()

data_raw_enc[cat_cols] = cbe.transform(data_raw_with_features_and_embed[cat_cols])
data_val_enc[cat_cols] = cbe.transform(data_val_with_features_and_embed[cat_cols])


In [66]:
data_raw_enc

,brand_name,CommercialTypeName4,resolution,desc_has_html_tags,desc_html_br_count,desc_html_p_count,desc_html_div_count,desc_html_span_count,desc_html_formatting_tags,desc_excessive_formatting,...,embed_90,embed_91,embed_92,embed_93,embed_94,embed_95,embed_96,embed_97,embed_98,embed_99
0,0.003009,0.001202,0,0,0,0.0,0.0,0.0,0,0,...,-2.895568e-09,2.644730e-09,-4.111773e-10,3.593262e-09,-7.055648e-09,-1.163583e-09,-5.886241e-10,-1.003650e-08,-4.453478e-09,-3.125168e-10
1,0.001068,0.001003,0,1,2,0.0,0.0,0.0,0,0,...,-6.039891e-09,-8.070578e-09,5.721585e-09,2.098877e-08,-2.607469e-08,3.559948e-09,-7.434722e-09,1.121490e-08,-1.049461e-08,-9.815407e-09
2,0.011031,0.000164,0,1,1,0.0,0.0,0.0,0,0,...,-2.052312e-09,-1.016610e-08,-1.818874e-10,9.253228e-09,1.211406e-08,-1.713271e-08,-3.662650e-09,-1.507357e-08,-2.553535e-08,-7.256180e-10
3,0.097664,0.000028,0,0,0,0.0,0.0,0.0,0,0,...,6.619181e-09,-4.312047e-10,-1.973761e-08,1.422100e-08,-2.706056e-08,2.737367e-09,-6.187059e-09,-5.925469e-09,-5.461460e-09,5.233232e-09
4,0.097664,0.000028,0,0,0,0.0,0.0,0.0,0,0,...,4.854918e-08,-4.230414e-08,2.206159e-08,-4.046312e-09,4.510063e-08,1.216446e-08,-5.226871e-08,-1.300305e-08,-1.979455e-08,-1.511037e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197193,0.013504,0.573606,0,0,0,0.0,0.0,0.0,0,0,...,3.797295e-03,2.283647e-02,-1.356090e-02,-5.524738e-02,2.509211e-02,-1.966623e-03,-2.216816e-03,-4.354743e-02,-1.057445e-02,3.047838e-04
197194,0.022844,0.006017,0,1,10,0.0,0.0,0.0,0,0,...,2.500488e-02,-5.930562e-03,-1.475624e-02,-1.269905e-02,1.861296e-02,3.957401e-03,9.528222e-03,-1.921244e-02,1.427810e-03,3.981487e-03
197195,0.250919,0.066187,0,1,2,0.0,0.0,0.0,0,0,...,4.523657e-02,1.156088e-02,-9.778230e-03,-1.034406e-02,3.506836e-04,3.088142e-02,2.777232e-02,-3.775661e-02,-1.664771e-02,-1.304939e-02
197196,0.034026,0.467145,1,1,6,0.0,0.0,0.0,0,0,...,5.490055e-02,9.018220e-03,7.299970e-02,3.175939e-02,-4.119861e-02,5.893227e-02,3.478235e-02,5.653106e-02,1.333023e-02,1.161287e-01


In [30]:
data_raw_prep , data_val_prep = pt.full_preprocess(path_train, path_test)


Initialized TF-IDF + SVD (fast mode)
Processing DataFrame with 197198 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 1972/1972 [00:37<00:00, 51.93it/s]


Created 100 embedding columns
Final DataFrame shape: (197198, 148)
Processing DataFrame with 22760 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 228/228 [00:04<00:00, 50.16it/s]


Created 100 embedding columns
Final DataFrame shape: (22760, 147)


0         0
1         0
2         0
3         0
4         0
         ..
197193    0
197194    0
197195    0
197196    1
197197    0
Name: resolution, Length: 197198, dtype: int64

In [42]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

In [68]:
X = data_raw_enc.drop('resolution', axis=1)
y = data_raw_enc['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.28, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


XGB_model = XGBClassifiermodel = XGBClassifier(
    n_estimators=120,
    max_depth=10,
    eval_metric="logloss", use_label_encoder=False,
    class_weight="balanced_subsample" ,
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])*0.3)# более мягкий вариант
)

Cat_model = CatBoostClassifier(
    iterations=1000,        # число деревьев (обычно 500–2000)
    learning_rate=0.05,     # темп обучения
    depth=8,                # глубина деревьев (6–10 норм)
    l2_leaf_reg=3,          # регуляризация
    random_seed=42,
    eval_metric="Recall",   # или AUC/Logloss/F1
    verbose=100             # печать каждые 100 итераций
)


In [69]:
XGB_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced_subsample', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None, ...)

In [76]:
y_pred = XGB_model.predict(X_test)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
f1_macro_RFC_model = f1_score(y_test, y_pred, average="macro"),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

(0.9703346855983773,)
              precision    recall  f1-score   support

           0       0.99      0.98      0.98     25780
           1       0.75      0.83      0.79      1828

    accuracy                           0.97     27608
   macro avg       0.87      0.90      0.89     27608
weighted avg       0.97      0.97      0.97     27608



In [77]:
confusion_matrix(y_test, y_pred)

array([[25280,   500],
       [  319,  1509]])

In [44]:
Cat_model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    use_best_model=True,    # модель вернётся на лучшей итерации
    early_stopping_rounds=100
)

0:	learn: 0.0290518	test: 0.0213465	best: 0.0213465 (0)	total: 190ms	remaining: 3m 9s
100:	learn: 0.2237948	test: 0.2129174	best: 0.2129174 (100)	total: 10.8s	remaining: 1m 36s
200:	learn: 0.3900181	test: 0.3672687	best: 0.3672687 (200)	total: 16.2s	remaining: 1m 4s
300:	learn: 0.4727041	test: 0.4378763	best: 0.4378763 (297)	total: 21.4s	remaining: 49.8s
400:	learn: 0.5342130	test: 0.4871374	best: 0.4887794 (398)	total: 26.6s	remaining: 39.7s
500:	learn: 0.5842290	test: 0.5276409	best: 0.5276409 (500)	total: 32s	remaining: 31.8s
600:	learn: 0.6130680	test: 0.5506294	best: 0.5511768 (596)	total: 37.1s	remaining: 24.6s
700:	learn: 0.6420134	test: 0.5747126	best: 0.5758073 (695)	total: 42.3s	remaining: 18s
800:	learn: 0.6720230	test: 0.5889436	best: 0.5889436 (797)	total: 47.4s	remaining: 11.8s
900:	learn: 0.6950090	test: 0.6009852	best: 0.6009852 (885)	total: 52.6s	remaining: 5.78s
999:	learn: 0.7119293	test: 0.6081007	best: 0.6081007 (996)	total: 57.7s	remaining: 0us

bestTest = 0.60810

In [46]:
y_pred = Cat_model.predict(X_test)
accuracy_CAT_model = accuracy_score(y_test, y_pred),
f1_macro_CAT_model = f1_score(y_test, y_pred, average="macro"),
report_CAT_model = classification_report(y_test, y_pred)
print(accuracy_CAT_model)
print(report_CAT_model)

(0.9665676615473776,)
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     25780
           1       0.85      0.60      0.71      1828

    accuracy                           0.97     27608
   macro avg       0.91      0.80      0.84     27608
weighted avg       0.96      0.97      0.96     27608



In [58]:
val_prediction = XGB_model.predict(data_val_prep)

In [59]:
val_prediction.sum()

np.int64(2102)

In [60]:
submission = pd.DataFrame({
    'id': data_val_full['id'],
    'prediction': val_prediction
    
})
submission.to_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv', index=False)
dddd = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv')
dddd.sum()

id            5060458544
prediction          2102
dtype: int64

In [87]:
data_prep_raw2, data_prep_val2 = pt.full_preprocess(path_train, path_test)

Initialized TF-IDF + SVD (fast mode)
Processing DataFrame with 197198 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 1972/1972 [00:25<00:00, 75.96it/s]


Created 100 embedding columns
Final DataFrame shape: (197198, 148)
Processing DataFrame with 22760 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 228/228 [00:03<00:00, 63.42it/s]


Created 100 embedding columns
Final DataFrame shape: (22760, 147)
Processing DataFrame with 197198 rows
Text column: name_rus
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 198/198 [00:03<00:00, 57.63it/s]


Created 100 embedding columns
Final DataFrame shape: (197198, 120)
Processing DataFrame with 22760 rows
Text column: name_rus
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 23/23 [00:00<00:00, 44.99it/s]


Created 100 embedding columns
Final DataFrame shape: (22760, 120)


In [171]:
X = data_prep_raw2.drop('resolution', axis=1)
y = data_prep_raw2['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.28, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


XGB_model2 = XGBClassifiermodel = XGBClassifier(
    n_estimators=120,
    max_depth=10,
    eval_metric="logloss", use_label_encoder=False,
    class_weight="balanced_subsample" ,
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])*0.3)# более мягкий вариант
)

Cat_model2 = CatBoostClassifier(
    iterations=1000,        # число деревьев (обычно 500–2000)
    learning_rate=0.05,     # темп обучения
    depth=8,                # глубина деревьев (6–10 норм)
    l2_leaf_reg=3,          # регуляризация
    random_seed=42,
    eval_metric="Recall",   # или AUC/Logloss/F1
    verbose=100             # печать каждые 100 итераций
)


In [172]:
XGB_model2.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced_subsample', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None, ...)

In [173]:
y_pred = XGB_model2.predict(X_test)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
f1_macro_RFC_model = f1_score(y_test, y_pred, average="macro"),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

(0.9732323964068386,)
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     25780
           1       0.78      0.83      0.80      1828

    accuracy                           0.97     27608
   macro avg       0.88      0.91      0.89     27608
weighted avg       0.97      0.97      0.97     27608



In [202]:
val_pred_proba_text

,0,1
0,0.999654,0.000346
1,0.999565,0.000435
2,0.999608,0.000392
3,0.999995,0.000005
4,0.988105,0.011895
...,...,...
22755,0.179117,0.820883
22756,0.999306,0.000694
22757,0.996730,0.003270
22758,0.999906,0.000094


In [139]:
metadata_raw = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/processed_data.csv')
metadata_val = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/processed_datatest.csv')

In [140]:
metadata_raw = metadata_raw.drop(['low_price_flag', 'ItemID', "name_rus","has_name","has_category","high_price_flag", 'id'],axis=1)
metadata_val = metadata_val.drop(['low_price_flag', 'ItemID', "name_rus","has_name","has_category","high_price_flag", 'id'],axis=1)

In [144]:
metadata_raw_clean, metadata_val_clean = pt.clean_small_brands(metadata_raw, metadata_val)

In [147]:
metadata_raw_clean_enc, metadata_val_clean_enc = pt.poluchenie_catboostenc(metadata_raw_clean, metadata_val_clean)

In [127]:
X = metadata_raw.drop('resolution', axis=1)
y = metadata_raw['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.28, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


XGB_model3 = XGBClassifiermodel = XGBClassifier(
    n_estimators=120,
    max_depth=10,
    eval_metric="logloss", use_label_encoder=False,
    class_weight="balanced_subsample" ,
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])*0.3)# более мягкий вариант
)

Cat_model3 = CatBoostClassifier(
    iterations=1000,        # число деревьев (обычно 500–2000)
    learning_rate=0.05,     # темп обучения
    depth=8,                # глубина деревьев (6–10 норм)
    l2_leaf_reg=3,          # регуляризация
    random_seed=42,
    eval_metric="Recall",   # или AUC/Logloss/F1
    verbose=100             # печать каждые 100 итераций
)
RFC_model3 = RandomForestClassifier(n_estimators=50, max_depth=10, class_weight="balanced", n_jobs=-1, random_state=42)


In [128]:
XGB_model3.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced_subsample', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None, ...)

In [129]:
y_pred = XGB_model3.predict(X_test)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
f1_macro_RFC_model = f1_score(y_test, y_pred, average="macro"),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

(0.9748985801217038,)
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     25780
           1       0.83      0.78      0.80      1828

    accuracy                           0.97     27608
   macro avg       0.91      0.88      0.90     27608
weighted avg       0.97      0.97      0.97     27608



In [209]:
X = metadata_raw_clean_enc.drop('resolution', axis=1)
y = metadata_raw_clean_enc['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.28, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


XGB_model4 = XGBClassifiermodel = XGBClassifier(
    n_estimators=120,
    max_depth=10,
    eval_metric="logloss", use_label_encoder=False,
    class_weight="balanced_subsample" ,
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])*0.3)# более мягкий вариант
)

Cat_model4 = CatBoostClassifier(
    iterations=1000,        # число деревьев (обычно 500–2000)
    learning_rate=0.05,     # темп обучения
    depth=8,                # глубина деревьев (6–10 норм)
    l2_leaf_reg=3,          # регуляризация
    random_seed=42,
    eval_metric="Recall",   # или AUC/Logloss/F1
    verbose=100             # печать каждые 100 итераций
)



In [210]:
XGB_model4.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced_subsample', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None, ...)

In [211]:
y_pred = XGB_model4.predict(X_test)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
f1_macro_RFC_model = f1_score(y_test, y_pred, average="macro"),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

(0.9781222833961171,)
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     25780
           1       0.83      0.84      0.83      1828

    accuracy                           0.98     27608
   macro avg       0.91      0.91      0.91     27608
weighted avg       0.98      0.98      0.98     27608



In [207]:
val_pred_proba_text = pd.DataFrame(XGB_model2.predict_proba(data_prep_val2))
raw_pred_proba_text = pd.DataFrame(XGB_model2.predict_proba(data_prep_raw2.drop('resolution', axis=1)))

In [212]:
val_pred_proba_meta = pd.DataFrame(XGB_model4.predict_proba(metadata_val_clean_enc))
raw_pred_proba_meta = pd.DataFrame(XGB_model4.predict_proba(metadata_raw_clean_enc.drop('resolution', axis=1)))

In [216]:

proba_meta_text_val = pd.DataFrame({
    'meta_proba_0': val_pred_proba_meta[0],
    'meta_proba_1': val_pred_proba_meta[1],
    'text_proba_0': val_pred_proba_text[0],
    'text_proba_1': val_pred_proba_text[1],
    
})

proba_meta_text_raw = pd.DataFrame({
    'meta_proba_0': raw_pred_proba_meta[0],
    'meta_proba_1': raw_pred_proba_meta[1],
    'text_proba_0': raw_pred_proba_text[0],
    'text_proba_1': raw_pred_proba_text[1],
    'resolution': data_prep_raw2['resolution']
    
})

In [217]:
proba_meta_text_val.to_csv('/Users/vladislavdemin/Desktop/ml_hack/proba_meta_text_val.csv', index=False)
proba_meta_text_raw.to_csv('/Users/vladislavdemin/Desktop/ml_hack/proba_meta_text_raw.csv', index=False)

In [158]:
val_prediction = Cat_model4.predict(metadata_val_clean_enc)

In [159]:
val_prediction.sum()

np.int64(1189)

In [154]:
submission = pd.DataFrame({
    'id': data_val_full['id'],
    'prediction': val_prediction
    
})
submission.to_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv', index=False)
dddd = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv')
dddd.sum()

id            5060458544
prediction          1399
dtype: int64

In [160]:
data_prep_raw3 = data_prep_raw2.drop(['brand_name', 'CommercialTypeName4', 'resolution'], axis=1)


In [162]:
data_prep_val3 = data_prep_val2.drop(['brand_name', 'CommercialTypeName4', ], axis=1)


In [163]:
conecting_data_raw = pd.concat([metadata_raw_clean_enc ,data_prep_raw3], axis=1) 
conecting_data_val = pd.concat([metadata_val_clean_enc ,data_prep_val3], axis=1) 

In [335]:
X = conecting_data_raw.drop('resolution', axis=1)
y = conecting_data_raw['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.28, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


XGB_model5 = XGBClassifier(
    n_estimators=150,
    max_depth=12,  # Уменьшаем глубину
    learning_rate=0.07587945476302646,  # Меньший learning rate
    min_child_weight=3,  # Увеличиваем для борьбы с переобучением
    subsample=1.0,  # Сэмплирование данных
    colsample_bytree=0.7,  # Сэмплирование признаков
    gamma=0.1,  # Минимальное уменьшение loss для разделения
    reg_alpha=0.01,  # L1 регуляризация
    reg_lambda=0.1,  # L2 регуляризация
    
    random_state=42
)



In [ ]:
 {'max_depth': 12, 'learning_rate': 0.07587945476302646, 'min_child_weight': 3, 'subsample': 1.0, 'colsample_bytree': 0.7, 'gamma': 0.1, 'reg_alpha': 0.01, 'reg_lambda': 0.1, 'n_estimators': 150}
 Optimal threshold: 0.685


In [336]:
XGB_model5.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0.1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07587945476302646, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=150, n_jobs=None,
              num_parallel_tree=None, ...)

In [338]:
threshold = 0.685
y_proba = XGB_model5.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= threshold).astype(int)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

(0.9760938858301942,)
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     25780
           1       0.91      0.71      0.80      1828

    accuracy                           0.98     27608
   macro avg       0.95      0.85      0.89     27608
weighted avg       0.98      0.98      0.97     27608



In [341]:
conecting_data_raw['resolutin'].sum()


KeyError: 'resolutin'

In [339]:
val_prediction = XGB_model5.predict(conecting_data_val)

In [340]:
val_prediction.sum()

np.int64(987)

In [346]:
submission = pd.DataFrame({
    'id': data_val_full['id'],
    'prediction': val_prediction
    
})
submission.to_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv', index=False)
dddd = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv')
dddd.sum()

id            5060458544
prediction           987
dtype: int64

In [220]:
from sklearn.linear_model import LogisticRegression

In [322]:
X = proba_meta_text_raw.drop('resolution', axis=1)
y = proba_meta_text_raw['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.28, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


XGB_model6 = XGBClassifiermodel = XGBClassifier(
    n_estimators=115,
    max_depth=11,
    min_child_weight=4,
    subsample=0.6044192289539582,
    colsample_bytree=0.8959108535080789,
    gamma=0.18039358968931873,
    reg_alpha=0.6910364993462792,
    reg_lambda=0.5679373312135461,
    eval_metric="logloss", use_label_encoder=False,
    class_weight="balanced_subsample" ,
    scale_pos_weight=10# более мягкий вариант
    
)



log_model6 = LogisticRegression(max_iter=1000, class_weight="balanced")

In [ ]:
{'n_estimators': 115, 'max_depth': 11, 'learning_rate': 0.03254703898212268, 'min_child_weight': 4, 'subsample': 0.6044192289539582, 'colsample_bytree': 0.8959108535080789, 'gamma': 0.18039358968931873, 'reg_alpha': 0.6910364993462792, 'reg_lambda': 0.5679373312135461}

In [309]:
XGB_model6.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8959108535080789, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, feature_weights=None,
              gamma=0.18039358968931873, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=4, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=115, n_jobs=None,
              num_parallel_tree=None, ...)

In [310]:
y_pred = XGB_model6.predict(X_test)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

(0.9787742683280208,)
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     25780
           1       0.87      0.80      0.83      1828

    accuracy                           0.98     27608
   macro avg       0.93      0.90      0.91     27608
weighted avg       0.98      0.98      0.98     27608



In [255]:

val_prediction2 = XGB_model6.predict(proba_meta_text_val)

In [256]:
val_prediction2.sum()

np.int64(1318)

In [257]:
log_model6.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

LogisticRegression(class_weight='balanced', max_iter=1000)

In [258]:
y_pred = log_model6.predict(X_test)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

(0.9796073601854535,)
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     25780
           1       0.85      0.84      0.85      1828

    accuracy                           0.98     27608
   macro avg       0.92      0.92      0.92     27608
weighted avg       0.98      0.98      0.98     27608



In [262]:
val_prediction = XGB_model6.predict(proba_meta_text_val)
submission = pd.DataFrame({
    'id': data_val_full['id'],
    'prediction': val_prediction
    
})
submission.to_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv', index=False)
dddd = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv')
dddd.sum()

id            5060458544
prediction          1318
dtype: int64

In [334]:
result = quick_optimize_xgboost(
        X_train, y_train, 
        X_val, y_val,
        n_trials_fast=15,    # Уменьшено для скорости
        n_trials_refined=5   # Уменьшено для скорости
    )
    
print(f"\nFinal results:")
print(f"Best F1 (default threshold): {result['f1_default']:.4f}")
print(f"Best F1 (optimized threshold): {result['f1_optimized']:.4f}")
print(f"Optimal threshold: {result['optimal_threshold']:.3f}")

[I 2025-08-22 00:29:42,121] A new study created in memory with name: no-name-8df4678c-bb91-4657-a736-3f8dbb3e5cc2


Optimizer initialized:
- Training samples: 141982
- Validation samples: 27608
- Optimization sample size: 10000
- Scale pos weight: 14.11

STAGE 1: Fast parameter search


Best trial: 0. Best value: 0.710546:   7%|▋         | 1/15 [00:39<09:12, 39.48s/it]

[I 2025-08-22 00:30:21,608] Trial 0 finished with value: 0.7105456324932713 and parameters: {'max_depth': 8, 'learning_rate': 0.01, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.2, 'reg_alpha': 0, 'reg_lambda': 0.1}. Best is trial 0 with value: 0.7105456324932713.


Best trial: 1. Best value: 0.724147:  13%|█▎        | 2/15 [00:52<05:10, 23.86s/it]

...

[I 2025-08-22 00:34:30,698] Trial 2 finished with value: 0.8143470573377115 and parameters: {'n_estimators': 100, 'max_depth': 12, 'learning_rate': 0.0641003510568888}. Best is trial 0 with value: 0.8242329693187728.


Best trial: 0. Best value: 0.824233:  80%|████████  | 4/5 [02:19<00:34, 34.35s/it]

[I 2025-08-22 00:35:06,563] Trial 3 finished with value: 0.8079503105590062 and parameters: {'n_estimators': 250, 'max_depth': 8, 'learning_rate': 0.0972918866945795}. Best is trial 0 with value: 0.8242329693187728.


Best trial: 0. Best value: 0.824233: 100%|██████████| 5/5 [03:10<00:00, 38.13s/it]


[I 2025-08-22 00:35:58,057] Trial 4 finished with value: 0.765869365225391 and parameters: {'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.02636424704863906}. Best is trial 0 with value: 0.8242329693187728.

Best F1 (refined): 0.8242
Final best params: {'max_depth': 12, 'learning_rate': 0.07587945476302646, 'min_child_weight': 3, 'subsample': 1.0, 'colsample_bytree': 0.7, 'gamma': 0.1, 'reg_alpha': 0.01, 'reg_lambda': 0.1, 'n_estimators': 150}

FINAL F1 Score: 0.8242

Threshold optimization:
- Default threshold (0.5) F1: 0.8242
- Optimal threshold: 0.685
- Optimal F1: 0.8342

Final results:
Best F1 (default threshold): 0.8242
Best F1 (optimized threshold): 0.8342
Optimal threshold: 0.685
